[View in Colaboratory](https://colab.research.google.com/github/cmajorsolo/machine_learning_python/blob/master/RNN_study_with_Tensorflow.ipynb)

In [7]:
import tensorflow as tf
x1 = tf.constant(5)
x2 = tf.constant(6)
# result = x1 * x2
result = tf.multiply(x1, x2)
print(result)

with tf.Session() as sess:
  output = sess.run(result)
print(output)


Tensor("Mul_9:0", shape=(), dtype=int32)
